In [1]:
import pandas as pd
import numpy as np

from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from matplotlib import pyplot
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.utils import to_categorical

import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import (
    precision_score, recall_score, roc_auc_score, f1_score, confusion_matrix
    )

Using TensorFlow backend.


## Preprocessing

In [2]:
quake_frame = pd.read_csv('data/consolidated_data.csv')

quake_frame['simple_label'] = quake_frame['type'] != 'earthquake'

quake_frame.drop(['id', 'Unnamed: 0', 'place', 'time', 'updated', 'type'], inplace=True, axis=1)

## Simple model, no imputation

We'll start this off with a simple model, just a Random Forest for two classes that takes only rows that have no nans in them. Let's see how many we get.  
Then we'll split the data 80/20 and run training.

In [3]:
quake_frame.dropna(inplace=True)
quake_frame.isna().sum()

latitude           0
longitude          0
depth              0
mag                0
magType            0
nst                0
gap                0
dmin               0
rms                0
net                0
horizontalError    0
depthError         0
magError           0
magNst             0
status             0
locationSource     0
magSource          0
simple_label       0
dtype: int64

In [4]:
len(quake_frame)

1227408

In [5]:
quake_frame.describe().transpose()

,count,mean,std,min,25%,50%,75%,max
latitude,1227408.0,37.361674,4.841731,0.000,35.964167,37.573000,38.817000,62.030667
longitude,1227408.0,-119.557707,10.027502,-179.098,-122.701333,-120.558833,-118.150167,179.661500
depth,1227408.0,6.016756,7.922880,-3.882,1.816000,4.413000,7.830000,211.000000
mag,1227408.0,1.258097,0.694405,-2.500,0.800000,1.180000,1.670000,5.840000
nst,1227408.0,17.010182,13.671235,0.000,8.000000,13.000000,22.000000,276.000000
gap,1227408.0,121.032150,65.767724,0.000,72.000000,105.000000,153.000000,360.000000
dmin,1227408.0,0.078264,0.342578,0.000,0.017120,0.037840,0.079990,141.160000
rms,1227408.0,0.097118,0.195847,0.000,0.030000,0.060000,0.130000,64.290000
horizontalError,1227408.0,0.801039,2.296862,0.000,0.270000,0.410000,0.720000,194.584100
depthError,1227408.0,2.773763,6.903563,0.000,0.490000,0.770000,1.460000,725.300000


Alright, this changes the proportions slightly, but not too bad. If anything, one might suggest that at least the mild increase in proportion of non-earthquakes offsets the reduced dataset a little.  
And the problematic values are no longer there, that's something.  
Let's try this.  
We'll start by mixing up the data frame, then encoding all the categories numerically and splitting it sklearn style.

In [6]:
quake_frame = quake_frame.sample(frac=1).reset_index(drop=True)

le = LabelEncoder()

cat_columns = ['magType', 'net', 'status', 'locationSource', 'magSource']

for cat in cat_columns:
    quake_frame[cat + '_enc'] = le.fit_transform(quake_frame[cat])

In [7]:
x_cols = ['latitude',
 'longitude',
 'depth',
 'mag',
 'nst',
 'gap',
 'dmin',
 'rms',
 'horizontalError',
 'depthError',
 'magError',
 'magNst',
 'magType_enc',
 'net_enc',
 'status_enc',
 'locationSource_enc',
 'magSource_enc']

y_col = ['simple_label']

### Try a Multilayer Perceptron undersampled

In [10]:
from imblearn.under_sampling import RandomUnderSampler
rus = RandomUnderSampler(random_state=42)
X, Y = rus.fit_resample(quake_frame[x_cols].values, quake_frame[y_col].values)

In [11]:
import pandas
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold

In [12]:
# Build custom metrics
from keras import backend as K

def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [14]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
metrics=['accuracy', precision_m, recall_m, f1_m]

cvscores = []
for train, test in kfold.split(X, Y):
  # create model
    model = Sequential()
    model.add(Dense(200, input_dim=len(x_cols), activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))    
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
    # Fit the model
    model.fit(X[train], Y[train], epochs=45, batch_size=20, verbose=15)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=15)
    print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))    
    cvscores.append(scores * 100)

Epoch 1/45
Epoch 2/45
Epoch 3/45
Epoch 4/45
Epoch 5/45
Epoch 6/45
Epoch 7/45
Epoch 8/45
Epoch 9/45
Epoch 10/45
Epoch 11/45
Epoch 12/45
Epoch 13/45
Epoch 14/45
Epoch 15/45
Epoch 16/45
Epoch 17/45
Epoch 18/45
Epoch 19/45
Epoch 20/45
Epoch 21/45
Epoch 22/45
Epoch 23/45
Epoch 24/45
Epoch 25/45
Epoch 26/45
Epoch 27/45
Epoch 28/45
Epoch 29/45
Epoch 30/45
Epoch 31/45
Epoch 32/45
Epoch 33/45
Epoch 34/45
Epoch 35/45
Epoch 36/45
Epoch 37/45
Epoch 38/45
Epoch 39/45
Epoch 40/45
Epoch 41/45
Epoch 42/45
Epoch 43/45
Epoch 44/45
Epoch 45/45
loss: 13.62%
accuracy: 94.94%
precision_m: 50.09%
recall_m: 46.98%
f1_m: 48.47%
Epoch 1/45
Epoch 2/45
Epoch 3/45
Epoch 4/45
Epoch 5/45
Epoch 6/45
Epoch 7/45
Epoch 8/45
Epoch 9/45
Epoch 10/45
Epoch 11/45
Epoch 12/45
Epoch 13/45
Epoch 14/45
Epoch 15/45
Epoch 16/45
Epoch 17/45
Epoch 18/45
Epoch 19/45
Epoch 20/45
Epoch 21/45
Epoch 22/45
Epoch 23/45
Epoch 24/45
Epoch 25/45
Epoch 26/45
Epoch 27/45
Epoch 28/45
Epoch 29/45
Epoch 30/45
Epoch 31/45
Epoch 32/45
Epoch 33/45
Ep

In [15]:
print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

0.51% (+/- 0.26%)


In [19]:
train_length = int(np.round(len(quake_frame.index) * 0.8))

In [20]:
train_X = quake_frame.loc[:train_length, x_cols]
train_y = quake_frame.loc[:train_length, y_col]

valid_X = quake_frame.loc[train_length:, x_cols]
valid_y = quake_frame.loc[train_length:, y_col]

## Try a Multilayer Perceptron oversampled

In [21]:
from imblearn.over_sampling import SMOTE

smoter = SMOTE(random_state=42)

X, Y = smoter.fit_resample(train_X.values, train_y.values)

In [22]:
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
metrics=['accuracy', precision_m, recall_m, f1_m]

cvscores = []
for train, test in kfold.split(X, Y):
  # create model
    model = Sequential()
    model.add(Dense(200, input_dim=len(x_cols), activation='relu'))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(60, activation='relu'))    
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=metrics)
    # Fit the model
    model.fit(X[train], Y[train], epochs=45, batch_size=20, verbose=15)
    # evaluate the model
    scores = model.evaluate(X[test], Y[test], verbose=15)
    scores_val = model.evaluate(valid_X, valid_y, verbose=15)
    print("%s: %.2f%%" % (model.metrics_names[0], scores[0]*100))
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores[3]*100))
    print("%s: %.2f%%" % (model.metrics_names[4], scores[4]*100))    
    print("%s: %.2f%%" % (model.metrics_names[0], scores_val[0]*100))
    print("%s: %.2f%%" % (model.metrics_names[1], scores_val[1]*100))
    print("%s: %.2f%%" % (model.metrics_names[2], scores_val[2]*100))
    print("%s: %.2f%%" % (model.metrics_names[3], scores_val[3]*100))
    print("%s: %.2f%%" % (model.metrics_names[4], scores_val[4]*100))    
    cvscores.append(scores * 100)
    cvscores.append(scores_val * 100)

Epoch 1/45
Epoch 2/45
Epoch 3/45
Epoch 4/45
Epoch 5/45
Epoch 6/45
Epoch 7/45
Epoch 8/45
Epoch 9/45
Epoch 10/45
Epoch 11/45
Epoch 12/45
Epoch 13/45
Epoch 14/45
Epoch 15/45
Epoch 16/45
Epoch 17/45
Epoch 18/45
Epoch 19/45
Epoch 20/45
Epoch 21/45
Epoch 22/45
Epoch 23/45
Epoch 24/45
Epoch 25/45
Epoch 26/45
Epoch 27/45
Epoch 28/45
Epoch 29/45
Epoch 30/45
Epoch 31/45
Epoch 32/45
Epoch 33/45
Epoch 34/45
Epoch 35/45
Epoch 36/45
Epoch 37/45
Epoch 38/45
Epoch 39/45
Epoch 40/45
Epoch 41/45
Epoch 42/45
Epoch 43/45
Epoch 44/45
Epoch 45/45
loss: 8.41%
accuracy: 97.09%
precision_m: 72.39%
recall_m: 80.45%
f1_m: 74.56%
loss: 21.96%
accuracy: 97.01%
precision_m: 47.18%
recall_m: 64.86%
f1_m: 52.54%
Epoch 1/45
Epoch 2/45
Epoch 3/45
Epoch 4/45
Epoch 5/45
Epoch 6/45
Epoch 7/45
Epoch 8/45
Epoch 9/45
Epoch 10/45
Epoch 11/45
Epoch 12/45
Epoch 13/45
Epoch 14/45
Epoch 15/45
Epoch 16/45
Epoch 17/45
Epoch 18/45
Epoch 19/45
Epoch 20/45
Epoch 21/45
Epoch 22/45
Epoch 23/45
Epoch 24/45
Epoch 25/45
Epoch 26/45
Epoch 2